In [13]:
import os
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
import pingouin as pg
import numpy as np

In [224]:
homedir = '/home/raghuram/Desktop/radiomics/TEXTURES/'
os.chdir(homedir)

In [15]:
t1ce_file = 'expt_t1ce.csv'
t1w_file = 'expt_t1w.csv'
t2f_file = 'expt_t2f.csv'
t2w_file = 'expt_t2w.csv'

In [151]:
def pre_process_dataframe(csv_file_name, experiment_number):
    # Binarize magnetization strength values
    # Scanner names and manufacturer are categorical 
    # One hot encode them
    
    # Code for the above here
    # Experiments in ascending order
    
    experiment_df = pd.read_csv(csv_file_name)
    experiment_df = experiment_df[experiment_df['experiment_number'] == experiment_number]
    fill_values = {'mag_field_strength':1.5}
    experiment_df.fillna(value=fill_values, inplace=True)
    experiment_df['mag_field_strength_binarized'] = (experiment_df['mag_field_strength']>=1.5).astype(int)
    experiment_df.drop(columns=[ 'Tumor','experiment_number', 'scale', 'algo', 'ng', 'flip_angle', 
                                'VOLUME_ET', 'VOLUME_NET', 'VOLUME_ED', 'VOLUME_TC',
                               'VOLUME_BRAIN', 'mag_field_strength'], inplace=True)
    
    experiment_df.dropna(subset=['VOLUME_WT'], inplace=True)
    experiment_df = pd.get_dummies(experiment_df, columns=['scanner_manufacturer', 'scanner_model'], drop_first=True)
    radiomics_features  = list(experiment_df.columns)[:42]

    radiomics_df = experiment_df[radiomics_features]
    scaled_df = experiment_df.drop(columns=radiomics_features)
    print(list(scaled_df.columns))
    scaled_df = scaled_df - scaled_df.min()/(scaled_df.max()-scaled_df.min())
    
    return (scaled_df, radiomics_df)
        


In [152]:
def results_summary_to_dataframe(results,response_variable):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    ci_bounds = results.conf_int()
 
    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "response_variable":response_variable,
                              })
    #Reordering...
    results_df = results_df[["coeff","pvals", "response_variable"]]
    return results_df

In [203]:
def linear_regression(results_folder, radiomic_df, scaled_df, experiment_number):
    # Load the data corresponding to the sequence argument here
    # Perform regression with Tr and Te 
    
    radiomic_features = list(radiomic_df.columns)
    y = np.array(radiomic_df)
    X = np.array(scaled_df)
    X = sm.add_constant(X)
    result_list = []
    for idx, column in enumerate(radiomic_features):
        
        model = sm.OLS(y[:, idx], X)
        results = model.fit()
        results_df = results_summary_to_dataframe(results, column)
#         results_df['significant'] = (results_df['pvals']<0.05).astype(bool)
        result_list.append(results_df)
       
    pd.concat(result_list).to_csv(os.path.join(results_folder, str(experiment_number)+'_'+'t1w.csv'), index=False)

In [225]:
results_folder = '/home/raghuram/Desktop/radiomics/TEXTURES/results/t1w/linear_regression'
for expt_number in range(1, 26):
    if expt_number > 1:
        break
    scaled_df, radiomics_df = pre_process_dataframe(t1w_file, expt_number)
    linear_regression(results_folder, radiomics_df, scaled_df, expt_number) 
    print('Finished experiment {}'.format(expt_number))

['repetition_time', 'excitation_time', 'VOLUME_WT', 'IDH1', '1p_19q_co_del_status', 'mag_field_strength_binarized', 'scanner_manufacturer_Philips', 'scanner_manufacturer_Siemens', 'scanner_model_Avanto', 'scanner_model_GENESIS', 'scanner_model_GENESIS_SIGNA', 'scanner_model_Ingenia', 'scanner_model_Intera', 'scanner_model_Intera Achieva', 'scanner_model_MAGNETOM VISION', 'scanner_model_SIGNA EXCITE', 'scanner_model_Signa HDxt', 'scanner_model_Symphony', 'scanner_model_Verio']
Finished experiment 1


In [227]:
results_folder = '/home/raghuram/Desktop/radiomics/TEXTURES/results/t1w/linear_regression'
os.chdir(results_folder)

In [196]:
csv_file_list = sorted(glob.glob('*.csv'), key=lambda _: int(_.split('_')[0]))

In [208]:
input_variables = ['intercept','repetition_time', 'excitation_time', 'VOLUME_WT', 'IDH1', '1p_19q_co_del_status', 'mag_field_strength_binarized', 'scanner_manufacturer_Philips', 'scanner_manufacturer_Siemens', 'scanner_model_Avanto', 'scanner_model_GENESIS', 'scanner_model_GENESIS_SIGNA', 'scanner_model_Ingenia', 'scanner_model_Intera', 'scanner_model_Intera Achieva', 'scanner_model_MAGNETOM VISION', 'scanner_model_SIGNA EXCITE', 'scanner_model_Signa HDxt', 'scanner_model_Symphony', 'scanner_model_Verio']
input_columns = input_variables*42
# for idx, csv_file in enumerate(csv_file_list):
#     df = pd.read_csv(csv_file)
# #     df['signifcant'] = (df['pvals']<0.05).astype(bool)
# #     df['input_variable'] = input_columns
#     df.set_index('response_variable', inplace=True)
#     if idx != 0:
#         df.drop(columns=['Unnamed: 0'], inplace=True)
#     df.to_csv(csv_file, index=False)
#     break

In [249]:
df = pd.read_csv('1_t1w.csv')
# df.drop(columns=['Unnamed: 0'], inplace=True)
# df['signifcant'] = (df['pvals']<0.05).astype(bool)
# df['input_variable'] = input_columns
# df = df.set_index(['response_variable', 'input_variable'], inplace=False)

# if idx != 0:
#     df.drop(columns=['Unnamed: 0'], inplace=True)
# df.to_csv(csv_file, index=False)


In [250]:
df = df.pivot_table(index='response_variable', columns='input_variable', values=['coeff', 'pvals'])

In [251]:
df

coeff                              \
input_variable     1p_19q_co_del_status          IDH1     VOLUME_WT   
response_variable                                                     
GLCM_Contrast              7.085538e-02 -2.542854e-01 -4.846333e-07   
GLCM_Correlation          -7.198097e-03  2.553227e-02  5.324581e-08   
GLCM_Dissimilarity         4.176178e-02 -1.066813e-01 -2.607308e-07   
GLCM_Energy               -3.484085e-03  6.475999e-03  2.128904e-08   
GLCM_Entropy               8.379105e-02 -1.821764e-01 -4.736099e-07   
GLCM_Homogeneity          -1.672435e-02  3.545217e-02  1.030848e-07   
GLCM_SumAverage            4.490460e-04 -2.023719e-04  4.252169e-10   
GLCM_Variance             -2.352361e-04  5.795289e-04  5.052522e-09   
GLRLM_GLN                 -7.057042e+03  2.684224e+04  1.625822e-02   
GLRLM_GLV                  1.126098e-04 -7.943794e-04 -4.744221e-09   
GLRLM_HGRE                -1.580953e-01 -5.217593e-02 -1.443062e-06   
GLRLM_LGRE                 1.883383e-03  3.634599e-03 -3.806380e-08   
GLRLM_LRE                 -3.584801e+00  4.328289e+00  2.147991e-05   
GLRLM_LRHGE               -8.846218e+01  1.107561e+02  6.285036e-04   
GLRLM_LRLGE               -1.258893e+00  1.042671e+00  7.651029e-06   
GLRLM_RLN                 -9.569437e+03  3.639847e+04  2.204622e-02   
GLRLM_RLV                  1.535359e-06 -2.752316e-05 -3.049621e-10   
GLRLM_RP                   3.179277e-02 -6.361641e-02 -2.259711e-07   
GLRLM_SRE                  3.445293e-02 -6.086412e-02 -1.994146e-07   
GLRLM_SRHGE                7.190563e-01 -1.567910e+00 -5.726599e-06   
GLRLM_SRLGE                3.249296e-03 -4.882268e-03 -4.443033e-08   
GLSZM_GLN                 -1.546327e+00  5.870885e+00  3.557857e-06   
GLSZM_GLV                  2.727320e-06 -4.272412e-06 -3.332620e-11   
GLSZM_HGZE                 7.844381e-01 -7.010175e-01 -4.851688e-06   
GLSZM_LGZE                -5.282997e-03  7.577827e-03  2.758129e-08   
GLSZM_LZE                 -2.151078e+07  7.876596e+06  4.345938e+02   
GLSZM_LZHGE               -5.252448e+08  2.148022e+08  1.122499e+04   
GLSZM_LZLGE               -4.853171e+06  1.503343e+06  7.552238e+01   
GLSZM_SZE                 -2.618408e-03 -2.986502e-02 -8.520256e-08   
GLSZM_SZHGE                1.959488e-01 -7.246538e-01 -4.179575e-06   
GLSZM_SZLGE               -5.429332e-04 -2.278164e-04  6.864960e-09   
GLSZM_ZP                   2.162799e-06 -1.173257e-03 -2.336872e-09   
GLSZM_ZSN                 -7.431897e-01  2.810525e+00  1.658267e-06   
GLSZM_ZSV                  1.242863e-01  2.616484e-03 -9.231695e-07   
Global_Kurtosis            5.843376e-01 -7.163675e-01  2.558214e-08   
Global_Skewness           -2.711237e-01  3.737408e-01  1.283507e-06   
Global_Variance           -6.176525e+00 -2.234050e+01 -2.474044e-04   
NGTDM_Busyness            -2.590986e+01  9.052568e+01  8.329101e-03   
NGTDM_Coarseness           8.260547e-05 -1.696996e-04 -1.276077e-09   
NGTDM_Complexity           1.700853e-01 -9.251089e-01 -1.854366e-06   
NGTDM_Contrast             1.197717e-03 -7.869946e-03 -1.357511e-08   
NGTDM_Strength             1.721674e-03 -3.568652e-03 -2.677966e-08   

                                                                               \
input_variable     excitation_time     intercept mag_field_strength_binarized   
response_variable                                                               
GLCM_Contrast         1.485449e-02  4.609472e-01                 2.078627e-01   
GLCM_Correlation     -1.527719e-03  5.455483e-01                 2.978584e-01   
GLCM_Dissimilarity    7.409831e-03  3.049122e-01                 1.269329e-01   
GLCM_Energy          -6.162125e-04  3.233604e-02                 2.180425e-02   
GLCM_Entropy          1.469876e-02  2.698455e+00                 1.365586e+00   
GLCM_Homogeneity     -2.799566e-03  4.588201e-01                 2.646963e-01   
GLCM_SumAverage       6.080742e-05  4.093191e-02                 2.222388e-02   
GLCM_Variance        -8.2067

In [256]:
df.xs('Global_Kurtosis')

       input_variable               
coeff  1p_19q_co_del_status             5.843376e-01
       IDH1                            -7.163675e-01
       VOLUME_WT                        2.558214e-08
       excitation_time                  1.221579e-01
       intercept                       -1.825431e-02
       mag_field_strength_binarized     3.492502e-02
       repetition_time                  4.910637e-05
       scanner_manufacturer_Philips     6.870013e-01
       scanner_manufacturer_Siemens    -2.985075e-01
       scanner_model_Avanto             5.109446e-01
       scanner_model_GENESIS           -7.743108e-01
       scanner_model_GENESIS_SIGNA      3.690463e-01
       scanner_model_Ingenia           -2.455002e+00
       scanner_model_Intera            -3.749239e-01
       scanner_model_Intera Achieva    -7.752709e-01
       scanner_model_MAGNETOM VISION   -5.317932e-02
       scanner_model_SIGNA EXCITE      -3.131971e-01
       scanner_model_Signa HDxt         3.117133e-01
       sc